# Nuclio - prediction function

## Setup the environment

In [4]:
# nuclio: ignore
import nuclio

### Set environment variables

In [19]:
# Iguazio access
%nuclio env FRAMESD=${V3IO_FRAMESD}
%nuclio env V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Model handling
%nuclio env MODEL_FILE=lgbm.model
%nuclio env -c MODEL_FILEPATH=/tmp/mlmodel/${MODEL_FILE}
%nuclio env -l MODEL_FILEPATH=models/trained/${MODEL_FILE}

# Function variables
%nuclio env NUM_OF_PRODUCTS_TO_RETURN=4
%nuclio env CUSTOMERS_TABLE=recommendation_demo_customers
%nuclio env PREDIOCTION_TABLE=recommendation_demo_recommendations

%nuclio: setting 'FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'MODEL_FILE' environment variable
%nuclio: setting 'MODEL_FILEPATH' environment variable
%nuclio: setting 'NUM_OF_PRODUCTS_TO_RETURN' environment variable
%nuclio: setting 'CUSTOMERS_TABLE' environment variable
%nuclio: setting 'PREDIOCTION_TABLE' environment variable


### Installations

In [18]:
%%nuclio cmd  
pip install json
pip install requests
pip install pandas
pip install lightgbm
pip install v3io_frames

  Could not find a version that satisfies the requirement json (from versions: )
No matching distribution found for json


### Imports

In [17]:
# Util
import os
import json
import requests

# Function
import pandas as pd
import lightgbm as lgb

# DB
import v3io_frames as v3f

## Function code

### Init context

In [22]:
def init_context(context):
    # define DB
    client = v3f.Client(os.environ['FRAMESD'])
    setattr(context, 'client', client)
    
    # define Model
    model = lgb.Booster(model_file="lgb.model")
    setattr(context, 'model', model)
    
    # vars
    setattr(context, 'k', int(os.environ['NUM_OF_PRODUCTS_TO_RETURN']))

In [29]:
def handler(context, event):
    # Get user parameters
    df = client.read('kv', )

    # Predict
    df["prediction"] = context.model.predict(df)

    # Eliminate low confidence results
    df = df.loc[df.prediction > 0.01, ['prediction', 'product_id']]

    # Keep top products
    df.sort_values(by='prediction', ascending=False, inplace=True)
    best_products = df.iloc[:context.k, :]


    return json.dumps(best_products.to_json())


def curl_server(command_func):
    def send_request(*args):
        v3io_function, table_name, key, payload, return_json = command_func(*args)

        server_url = os.getenv('SERVER_URL', '')
        url = "{}/1/{}/{}".format(server_url, table_name, key)

        headers = {
            "Content-Type": "application/json",
            "X-v3io-function": v3io_function,
            "Connection": "close",
        }
        try:
            response = requests.post(url, json=payload, headers=headers, timeout=5)
            response.close()
        except:
            return {"item": {"products": {"S": ""}}}

        if return_json == True:
            return response.json()
        else:
            if response.status_code == 200 or response.status_code == 204:
                return True
            else:
                return False

    return send_request


@curl_server
def update_customer_location(customer_id, location, return_json=False):
    v3io_function = "UpdateItem"
    table_name = "customers"
    payload = {
        "UpdateExpression": f"SET location='{location}'"
    }

    return v3io_function, table_name, customer_id, payload, return_json


@curl_server
def get_customer(customer_id: str, return_json=True):
    v3io_function = "GetItem"
    table_name = "customers"
    payload = {"AttributesToGet": "*"}

    return v3io_function, table_name, customer_id, payload, return_json


def response_to_event(data):
    X: dict = json.loads(data["Item"]["products"]["S"])
    return X